In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# Read the data
X = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv')

In [3]:
# Remove rows with missing target, separate target from previous one
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(
                                            X, y, train_size=0.8, test_size=0.2,random_state=0)

In [5]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test  = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test  = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test  = X_train.align(X_test  , join='left', axis=1)


In [6]:
# Define the model
model_1 = XGBRegressor(random_state=0)

# Fit the model
model_1.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [7]:
# Get predictions
predictions_1 = model_1.predict(X_valid)
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1,y_valid)
print("Mean Absolute Error: " , mae_1)

Mean Absolute Error:  17662.736729452055


In [8]:
# Define the model
model_2 = XGBRegressor(n_estimator=1000, learning_rate=0.05)

# Fit the model
model_2.fit(X_train,y_train)



[16:51:56] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "n_estimator" } are not used.



XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimator=1000, n_estimators=100,
             n_jobs=0, num_parallel_tree=1, predictor='auto', ...)

In [9]:
# Get predictions
predictions_2 = model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2,y_valid)
print("Mean Absolute Error: " , mae_2)

Mean Absolute Error:  17213.82253317637


In [10]:
# Define the model
model_3 = XGBRegressor(n_estimators=100,learning_rate=0.01)

# Fit the model
model_3.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [11]:
# Get predictions
predictions_3 = model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)
print("Mean Absolute Error: ", mae_3)

Mean Absolute Error:  68584.79109589041


In [12]:
print("MAE 1: " , mae_1 , "(Using random_state = 0)")
print("MAE 2: " , mae_2 , "(Using n_estimators  = 1000, learning_rate = 0.05)")
print("MAE 3: " , mae_3 , "(Using n_estimators  = 100 , learning_rate = 0.01)")

MAE 1:  17662.736729452055 (Using random_state = 0)
MAE 2:  17213.82253317637 (Using n_estimators  = 1000, learning_rate = 0.05)
MAE 3:  68584.79109589041 (Using n_estimators  = 100 , learning_rate = 0.01)
